# Test pyspotstream

* This is a simple test to check whether:
  * `pyspotstream` is installed
  * the `OPM` data set can be downloaded to the local disk
  * If successful, data can be loaded and the CSV file `open_data.csv` is available.

In [31]:
from pyspotstream.data.OPM import get_opm, load_opm

## Download OPM Data

In [32]:
get_opm(overwrite=False)

File opm_data.csv already exists. To enforce download, set 'overwrite=True'.


## Load OPM

In [33]:
X_train, y_train = load_opm(data_type="num", n=10000, sorted=True, verbose=False)
y_train = y_train.squeeze()  # necessary because Y is a one column df
X_train


/Users/bartz/miniforge3/envs/spotCondaEnv/lib/python3.10/site-packages/pyspotstream/data/OPM.py:43: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, date_parser=dateparse)


,List Year,Assessed Value,Sales Ratio,timestamp_rec,lat,lon
0,2001,107530.0,0.575027,1001887200,-1.000000,-1.000000
1,2001,47000.0,0.653686,1001887200,-1.000000,-1.000000
2,2001,50720.0,0.878268,1001887200,-1.000000,-1.000000
3,2001,80010.0,0.695739,1001887200,-72.441559,41.867161
4,2001,152800.0,0.485079,1001887200,-1.000000,-1.000000
...,...,...,...,...,...,...
9995,2001,378840.0,0.557938,1007420400,-1.000000,-1.000000
9996,2001,56800.0,0.541468,1007420400,-1.000000,-1.000000
9997,2001,84660.0,0.617956,1007420400,-1.000000,-1.000000
9998,2001,37520.0,0.487273,1007420400,-73.225151,41.489830
